# Data Preparation

## MovieLens Data

With the data now formatted, read the CSV files back into Pandas.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reading ratings file
# Ignore the timestamp column
ratings = pd.read_csv('ratings.csv', encoding='latin-1', usecols=['user_id', 'movie_id', 'rating'])

# Reading users file
users = pd.read_csv('users.csv', sep='\t',encoding='latin-1', usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])

# Reading movies file
movies = pd.read_csv('movies.csv', sep='\t',encoding='latin-1', usecols=['movie_id', 'title', 'genres'])

### Feature Alignment

Review the head of the ratings dataset.

In [3]:
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [4]:
#note that film_ids are 1-indexed
ratings.user_id.min() == ratings.movie_id.min() == movies.movie_id.min() == 1

True

Since the user ID and the film ID are currently indexed at 1, we reduce the values by 1 to make the data features 0-indexed - this will allow us to analyze the data and use row index as a proxy for the index value in the underlying dataset.

In [5]:
#reindex user_id so that it is zero-indexed
ratings['user_id'] = ratings.user_id - 1

#reindex film_id for same reason
ratings['movie_id'] = ratings.movie_id - 1
movies['movie_id'] = movies.movie_id - 1

In [6]:
#confirm processing
ratings.user_id.min() == ratings.movie_id.min() == movies.movie_id.min() == 0

True

There are ratings for movies not contained within the movie database - we find and drop these values from the review database.

In [7]:
#check count
ratings.shape

(1000209, 3)

In [8]:
from tqdm import tqdm_notebook
for index, movie_id in tqdm_notebook(enumerate(ratings.movie_id)):
    if movie_id in movies.movie_id:
        pass
    else:
        ratings.drop(index, inplace = True, axis = 0)   

In [9]:
#check count again
ratings.shape

(991384, 3)

Also, the movie IDs are not consecutive values - we adjust this by capturing the original index-value pairs of the movie database into a dictionary and using that dictionary to re-index the values in the reviews database.  This allows us to drop the ID columns and simply use the row index as the identifier in our future analysis.

In [10]:
movies.movie_id.tail()

3878    3947
3879    3948
3880    3949
3881    3950
3882    3951
Name: movie_id, dtype: int64

In [11]:
def DatasetIndexExtract(dfcol):
    '''
    Accepts a column of a DataFrame.  Extracts the value and DataFrame index of that column,
    returning a dictionary of {value: index} that can be used to transform the index of the dataset to
    match the row indexing of the DataFrame.
    '''
    index_dict = {}
    for index, value in enumerate(dfcol):
        index_dict.update({value : index})
    return index_dict

In [12]:
#extract index_dict
index_dict = DatasetIndexExtract(movies.movie_id)

#apply over movies database
tqdm_notebook(movies.movie_id.replace(to_replace = index_dict, value = None, inplace = True))

0/|/| 0/? [00:00<?, ?it/s]

In [13]:
#confirm dataset indexing
movies.movie_id.tail()

3878    3878
3879    3879
3880    3880
3881    3881
3882    3882
Name: movie_id, dtype: int64

With the movie ID transformed within the movies dataset, we apply the same transformation to the ratings index.

In [14]:
ratings.movie_id.replace(to_replace = index_dict, value = None, inplace = True)

While we initially attempted to reindex the datasets using a helper function...

```Python
from tqdm import tqdm_notebook

def DatasetReindex(dfcol,index_dict):
    '''
    Accepts a DataFrame column and a dictionary of {value:index} pairs.  Transforms the column
    to match the indexing of the underlying DataFrame.
    '''
    for index, value in tqdm_notebook(enumerate(dfcol)):
        try:
            dfcol.iloc[index] = index_dict[value]
        except:
            print("error at index ", index)
```
...as always, the general principle of avoiding iterating over a Pandas DataFrame applied: the function above was estimating about 9hr to completion, whereas pd.DataFrame.replace completed the job in mere seconds.

Once the process has efficiently completed, we confirm that the re-indexing has achieved its goal by passing through the movie IDs in the ratings database and verifying they are all present in the movies database.  Given that we need a verbose output and that no transformation of the variables is required, iterating over the DataFrame is a reasonable approach.

In [15]:
for movie_id in tqdm_notebook(ratings.movie_id):
    if movie_id in movies.movie_id:
        pass
    else:
        print("{0} not in movies database".format(movie_id)) 

As hoped, no errors were generated.

With the movies database now reindexed to align with the "native" indexing inherent to a Pandas DataFrame, we now transform the values for year and genre so that they can be machine processed.

In [16]:
#review current format
movies.head()

,movie_id,title,genres
0,0,Toy Story (1995),Animation|Children's|Comedy
1,1,Jumanji (1995),Adventure|Children's|Fantasy
2,2,Grumpier Old Men (1995),Comedy|Romance
3,3,Waiting to Exhale (1995),Comedy|Drama
4,4,Father of the Bride Part II (1995),Comedy


We create a series of indicator variables for each genre present in the current column.

In [17]:
#count number of genre references in original list for debugging
movies[movies.genres.str.contains("Drama")].shape

(1603, 3)

In [18]:
#separate genre values by row, format as a list, and save into DF column
movies['genre_list'] = movies['genres'].str.split('|').values  

#build indicator variables from list values
movies['is_drama'] = movies.genre_list.apply(lambda x: 1 if 'Drama' in x else 0)
movies['is_comedy'] = movies.genre_list.apply(lambda x: 1 if 'Comedy' in x else 0)
movies['is_action'] = movies.genre_list.apply(lambda x: 1 if 'Action' in x else 0)
movies['is_thriller'] = movies.genre_list.apply(lambda x: 1 if 'Thriller' in x else 0)
movies['is_romance'] = movies.genre_list.apply(lambda x: 1 if 'Romance' in x else 0)
movies['is_horror'] = movies.genre_list.apply(lambda x: 1 if 'Horror' in x else 0)
movies['is_adventure'] = movies.genre_list.apply(lambda x: 1 if 'Adventure' in x else 0)
movies['is_scifi'] = movies.genre_list.apply(lambda x: 1 if 'Sci-Fi' in x else 0)
movies['is_childrens'] = movies.genre_list.apply(lambda x: 1 if "Children's" in x else 0)
movies['is_crime'] = movies.genre_list.apply(lambda x: 1 if 'Crime' in x else 0)
movies['is_war'] = movies.genre_list.apply(lambda x: 1 if 'War' in x else 0)
movies['is_documentary'] = movies.genre_list.apply(lambda x: 1 if "Documentary" in x else 0)
movies['is_musical'] = movies.genre_list.apply(lambda x: 1 if 'Musical' in x else 0)
movies['is_mystery'] = movies.genre_list.apply(lambda x: 1 if 'Mystery' in x else 0)
movies['is_animation'] = movies.genre_list.apply(lambda x: 1 if "Animation" in x else 0)
movies['is_fantasy'] = movies.genre_list.apply(lambda x: 1 if 'Fantasy' in x else 0)
movies['is_western'] = movies.genre_list.apply(lambda x: 1 if 'Western' in x else 0)
movies['is_filmnoir'] = movies.genre_list.apply(lambda x: 1 if "Film-Noir" in x else 0)

#drop the column containing lists of genres
movies.drop('genre_list', inplace = True, axis = 1)
movies.drop('genres', inplace = True, axis = 1)

In [19]:
#confirm processing - is_drama should be same count as before
movies.is_drama.sum()

1603

In [20]:
#review transformed data
movies.head()

,movie_id,title,is_drama,is_comedy,is_action,is_thriller,is_romance,is_horror,is_adventure,is_scifi,is_childrens,is_crime,is_war,is_documentary,is_musical,is_mystery,is_animation,is_fantasy,is_western,is_filmnoir
0,0,Toy Story (1995),0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1,1,Jumanji (1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0
2,2,Grumpier Old Men (1995),0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,Waiting to Exhale (1995),1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,Father of the Bride Part II (1995),0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


With the genres now extracted, we pull out the year values from the movie title and place it into a seperate column.

In [21]:
import re

#create temporary column holding list of every space-separated word in title
movies['year_extraction'] = movies.title.str.split()

#initalize and apply function to strip () from year
table = str.maketrans(dict.fromkeys("()"))
movies['year'] = movies.year_extraction.apply(lambda x: x[-1].translate(table))

#there is one movie with a set of () at the end of the title - manually correct year value
movies.year.loc[movies.year == "L'Associe1982"] = 1982

#drop temporary column
movies.drop('year_extraction', inplace = True, axis = 1)

#remove year data from movie name
movies['title'] = movies.title.apply(lambda x: re.sub(r'\(.*\)', '', x))

#strip trailing space left after year values removed
movies['title'] = movies.title.apply(lambda x: x.strip())

C:\Users\cahib\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [22]:
#confirm extraction
movies.head()

,movie_id,title,is_drama,is_comedy,is_action,is_thriller,is_romance,is_horror,is_adventure,is_scifi,...,is_crime,is_war,is_documentary,is_musical,is_mystery,is_animation,is_fantasy,is_western,is_filmnoir,year
0,0,Toy Story,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1995
1,1,Jumanji,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1995
2,2,Grumpier Old Men,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995
3,3,Waiting to Exhale,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1995
4,4,Father of the Bride Part II,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1995


In [23]:
#convert year values from str to float
movies['year'] = movies.year.astype(int)

#review values
movies.year.describe()

count    3883.000000
mean     1986.066959
std        16.895690
min      1919.000000
25%      1982.000000
50%      1994.000000
75%      1997.000000
max      2000.000000
Name: year, dtype: float64

We now turn our attention to the user data, and start by reviewing the data as currently formatted.

In [34]:
#review users DF
users.head()

,user_id,gender,zipcode,age_desc,occ_desc
0,1,F,48067,Under 18,K-12 student
1,2,M,70072,56+,self-employed
2,3,M,55117,25-34,scientist
3,4,M,02460,45-49,executive/managerial
4,5,M,55455,25-34,writer


In [35]:
#review data formatting
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
user_id     6040 non-null int64
gender      6040 non-null object
zipcode     6040 non-null object
age_desc    6040 non-null object
occ_desc    6040 non-null object
dtypes: int64(1), object(4)
memory usage: 236.0+ KB


In [36]:
#check for null values
users.isnull().any()

user_id     False
gender      False
zipcode     False
age_desc    False
occ_desc    False
dtype: bool

Again, since the user_id values are 1-indexed in the original dataset, we reindex to Python's native 0-index format by subtracting one from the current values.

In [37]:
users.user_id.min()

1

In [28]:
users['user_id'] = users.user_id - 1

In [38]:
users.user_id.min()

1

With that basic transformation complete, we begin to engineer the features into a machine-readable format.  First, we will reclassify the gender values into two indicator variables.

In [39]:
# Classify gender
gender_dummies = pd.get_dummies(users['gender'], prefix = "is")

#drop gender col
users.drop('gender', inplace = True, axis = 1)

#add values in
users = pd.concat([users, gender_dummies],axis = 1).copy()

In a perfect world, we could get very granular with the location info, but given that we only have 6040 users in this dataset, we will reduce the ZIP code down to the first digit.  Since this first digit represents a given geographic region, we can use this feature to identify films that are more popular than average in a given region.

In [40]:
# Get region info from zipcode
users['zip'] = users['zipcode'].astype(str).str[0]

#drop zipcode col
users.drop('zipcode', inplace = True, axis = 1)

#create indicator variables
zip_dummies = pd.get_dummies(users['zip'], prefix = "zip")

#drop zip col
users.drop('zip', inplace = True, axis = 1)

In [ ]:
users = pd.concat([users, gender_dummies],axis = 1).copy()

We now turn our attention to the age values.  The dataset conveniently already bundled the ages into nice demographic groupings, so we simply need to transform these values into indicator variables.  First, however, we will reparse these values to make them more Python-friendly.

In [ ]:
users.age_desc.unique()

In [ ]:
age_dict = {'Under 18':'u18', '56+': '56plus', '25-34': '25_34', '45-49':'45_49', '50-55':'50_55', '35-44':'35_44', '18-24':'18_24'}

In [ ]:
users.age_desc.replace(to_replace = age_dict, value = None, inplace = True)

In [ ]:
users.age_desc.unique()

In [ ]:
# Classify age groups
age_dummies = pd.get_dummies(users['age_desc'], prefix = 'age')
age_dummies.head()

In [ ]:
#add age cols into main DF
users = pd.concat([users, age_dummies],axis = 1).copy()

We turn finally to the occupation data.  As we did with the age groups, we first reparse the names and then convert them into indicator variables.

In [ ]:
users['occ_desc'].value_counts()

In [ ]:
users['occ_desc'].unique()

In [ ]:
job_dict = {'K-12 student':'k12student', 'self-employed':'self_emp', 
       'executive/managerial':'exec', 'academic/educator':'teacher',
        'technician/engineer':'engineer', 'other or not specified':'other',
       'clerical/admin':'clerical', 'sales/marketing':'sales', 'college/grad student':'college_student',
       'tradesman/craftsman':'trade','customer service':'cust_service',  'doctor/health care':'medical'}

In [ ]:
users.occ_desc.replace(to_replace = job_dict, value = None, inplace = True)

In [ ]:
users.occ_desc.unique()

In [ ]:
# Classify job groups
occ_dummies = pd.get_dummies(users['occ_desc'], prefix ='job')
occ_dummies.head()

In [ ]:
users = pd.concat([users, occ_dummies],axis = 1).copy()

Now that all the features within the users dataset have been transformed, we make a final check of the file and get a sense of the distribution among the various features.

In [ ]:
users.mean()

With the data preparation now complete, save the prepared files back into CSV.

In [ ]:
# Save into movies_prepared.csv
movies.to_csv("movies_prepared.csv", header=True)

# Save into users_prepared.csv
users.to_csv("users_prepared.csv", header=True, index = False) 

# Save into ratings_prepared.csv
ratings.to_csv("ratings_prepared.csv", header=True, index = False)

## TMDB 5000 Data

Moving into the TMDB dataset, we again begin by loading the raw data.

In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from ast import literal_eval
import datetime as dt

import os
os.chdir('C:\\Users\\cahib\\Downloads\\Data Mining_MSCA_31008\\Final Project\\movielens-master\\dat')
os.getcwd()

In [ ]:
tmdb_credits=pd.read_csv('tmdb_5000_credits.csv')
tmdb_movies=pd.read_csv('tmdb_5000_movies.csv')

With the data loaded, we review the raw format.

In [ ]:
tmdb_credits.head(3)

In [ ]:
tmdb_movies.head(2)

We merge these two DataFrames into a single movies dataset.

In [ ]:
tmdb_credits.columns = ['id','tittle','cast','crew']
tmdb_movies= tmdb_movies.merge(tmdb_credits,on='id')

#confirm merge
tmdb_movies.head(2)

We can drop many of these columns as either duplicates or as unnecessary for our analysis.

In [ ]:
tmdb_movies.drop('homepage', inplace = True, axis = 1)
tmdb_movies.drop('tittle', inplace = True, axis = 1)
tmdb_movies.drop('original_language', inplace = True, axis = 1)
tmdb_movies.drop('spoken_languages', inplace = True, axis = 1)
tmdb_movies.drop('original_title', inplace = True, axis = 1)
tmdb_movies.drop('status', inplace = True, axis = 1)
tmdb_movies.drop('production_countries', inplace = True, axis = 1)

#confirm file status
tmdb_movies.head(2)

We can now begin preparing the data for machine processing.  As a glance at the underlying data and a review of the typing imputed by Pandas shows, many of the fields will need to be transformed.

In [ ]:
tmdb_movies.info()

We begin by converting the release date and year values into DateTimes.

In [ ]:
#transform release date to DateTime format
tmdb_movies['release_date'] = pd.to_datetime(tmdb_movies['release_date'],errors='coerce')

#create a year column
tmdb_movies['year'] = pd.to_datetime(tmdb_movies['release_date'],errors='coerce').dt.year

#purge NA values
tmdb_movies = tmdb_movies.dropna(subset=['year'])

In [ ]:
#review year values
tmdb_movies.year.head(3)

In [ ]:
#review release_date values
tmdb_movies.release_date.head(3)